In [4]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import sys
sys.path.append("..") 
from Portfolio.portfolio_tools import *

import tink_port as tink

/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Вводим токен, получаем список доступных аккаунтов

In [5]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'

token = TOKEN1

In [6]:
accs = tink.get_accounts(token)
accs

GetAccountsResponse(accounts=[Account(id='2191186267', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='ETF', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2022, 7, 29, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>)])

In [1]:
from tinkoff.invest import Client
from tinkoff.invest.constants import INVEST_GRPC_API, INVEST_GRPC_API_SANDBOX




with Client(TOKEN, target=INVEST_GRPC_API) as client:
    print(client.users.get_accounts())
    

NameError: name 'token' is not defined

#### База идентификаторов тиньков

In [3]:
base = tink.get_id_base(token)
port = tink.get_portfolio(token)
df_port = tink.port_to_df(port, base)

In [4]:
df_port

,figi,ticker,name,quantity,price
3,BBG007N0Z367,AGRO,РусАгро,16,1417.200
8,BBG004S686N0,BANEP,Башнефть - привилегированные акции,14,2116.000
13,BBG00475K6C3,CHMF,Северсталь,39,1883.800
0,BBG004731032,LKOH,ЛУКОЙЛ,10,7493.000
5,BBG004S68507,MAGN,Магнитогорский металлургический комбинат,740,56.590
1,BBG004S681B4,NLMK,НЛМК,490,222.600
11,TCS00A103X66,POSI,Positive Technologies,11,2716.800
14,BBG004731354,ROSN,Роснефть,37,555.400
2,TCS00A107RZ0,RU000A107RZ0,ГК Самолет выпуск 13,40,993.300
12,BBG004730N88,SBER,Сбер Банк,220,295.100


### Load account data

In [93]:
res = []

for pos in port.positions:
    print(pos.figi)        
    candles = tink.get_candles(token, pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  tink.get_open_price(candles)
    ticker = tink.figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)

BBG004731032
BBG004S681B4
TCS00A107RZ0
BBG007N0Z367
BBG00475KHX6
BBG004S68507
BBG004RVFFC0
BBG004S684M6
BBG004S686N0
BBG004S681M2
RUB000UTSTOM
TCS00A103X66
BBG004730N88
BBG00475K6C3
BBG004731354


In [94]:
dfp = dfp.dropna(axis = 1)
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


### Portfolio Black-Litterman Model

In [8]:
mcaps = {}
for t in dfp.columns:
   # mcaps[t] = stock.info["marketCap"]
    mcaps[t] = 1
mcaps

{'LKOH': 1,
 'NLMK': 1,
 'AGRO': 1,
 'MAGN': 1,
 'TATN': 1,
 'SIBN': 1,
 'BANEP': 1,
 'SNGSP': 1,
 'POSI': 1,
 'SBER': 1,
 'CHMF': 1,
 'ROSN': 1}

In [9]:
mcaps['SBER'] = 1.5
mcaps['POSI'] = 1.5

In [28]:
imoex_figi = 'BBG333333333'
market_prices = tink.get_candles(token, imoex_figi, CandleInterval.CANDLE_INTERVAL_DAY)
market_prices =  tink.get_open_price(market_prices)

In [21]:
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


In [38]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel

S = risk_models.CovarianceShrinkage(dfp).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices['ticker'])
delta

9.802403162030824

In [39]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

LKOH     0.235540
NLMK     0.250621
AGRO     0.404671
MAGN     0.222163
TATN     0.175982
SIBN     0.177426
BANEP    0.306354
SNGSP    0.265854
POSI     0.254061
SBER     0.201803
CHMF     0.241332
ROSN     0.239506
dtype: float64

In [41]:
# You don't have to provide views on all the assets
viewdict = {
    "POSI": 0.10,
}

confidences = [
    0.6
]
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

In [43]:
intervals = [
    (0.1, 0.4),
]

variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

[0.022500000000000006]


In [44]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [45]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl

LKOH     0.210907
NLMK     0.225580
AGRO     0.377385
MAGN     0.198196
TATN     0.153239
SIBN     0.155086
BANEP    0.281036
SNGSP    0.241803
POSI     0.213216
SBER     0.178689
CHMF     0.218008
ROSN     0.216051
dtype: float64

In [46]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
LKOH,0.235540,0.210907,NaN
NLMK,0.250621,0.225580,NaN
AGRO,0.404671,0.377385,NaN
MAGN,0.222163,0.198196,NaN
TATN,0.175982,0.153239,NaN
SIBN,0.177426,0.155086,NaN
BANEP,0.306354,0.281036,NaN
SNGSP,0.265854,0.241803,NaN
POSI,0.254061,0.213216,0.1
SBER,0.201803,0.178689,NaN


In [49]:
S_bl = bl.bl_cov()


In [50]:
from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights
     

D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


OrderedDict([('LKOH', 0.07897),
             ('NLMK', 0.08567),
             ('AGRO', 0.14173),
             ('MAGN', 0.07393),
             ('TATN', 0.05579),
             ('SIBN', 0.0562),
             ('BANEP', 0.10686),
             ('SNGSP', 0.09078),
             ('POSI', 0.08075),
             ('SBER', 0.06611),
             ('CHMF', 0.08197),
             ('ROSN', 0.08124)])

In [52]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, dfp.iloc[-1], total_portfolio_value=100000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

Leftover: $703.29


D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


{'LKOH': 1,
 'NLMK': 39,
 'AGRO': 10,
 'MAGN': 130,
 'TATN': 8,
 'SIBN': 8,
 'BANEP': 5,
 'SNGSP': 141,
 'POSI': 3,
 'SBER': 22,
 'CHMF': 4,
 'ROSN': 15}

### Model

In [56]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

In [62]:

def calc_frontier(df_period, risk_method, ret_method = "mean_historical_return", span = 180):

    if ret_method == "ema_historical_return":
        mu = expected_returns.return_model(df_period, method=ret_method, span = span)
    else:
        mu = expected_returns.return_model(df_period, method=ret_method)
  
    cov_mat = risk_models.risk_matrix(df_period, method=risk_method)        
    ef = EfficientFrontier(mu, cov_mat)
    return ef


def calc_weights(ef, opt_type, par, verbose = False):

    if opt_type == 'max_sharpe':
        try:
            weights = ef.max_sharpe()
        except:
            if verbose:
                print("Non-convex optimize!")
            weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
        )  
    elif opt_type == 'efficient_risk':
        weights = ef.efficient_risk(par) 

    ef.portfolio_performance(verbose=verbose)
    cleaned_weights = ef.clean_weights(cutoff=0.0001)
    dfw = weights_to_df(cleaned_weights)
    dfw['W'] = dfw['W']/dfw['W'].sum()
    return dfw


In [95]:
drops = []

columns = [x for x in dfp.columns if x not in drops]
dfpx = dfp[columns]

In [109]:
#ef = calc_frontier(dfp, "semicovariance",  "ema_historical_return", span = 90)
ef = calc_frontier(dfpx, "ledoit_wolf",  "ema_historical_return", span = 180)
#ef = calc_frontier(dfp, "ledoit_wolf_constant_correlation",  "mean_historical_return")
#ef = calc_frontier(dfp, "exp_cov",  "capm_return")
 
#ef = calc_frontier(dfp, "ledoit_wolf_constant_variance",  "mean_historical_return")
#ef = calc_frontier(dfp, "sample_cov")
#weights = ef.max_sharpe()


dfw = calc_weights(ef, 'max_sharpe', 0, verbose = True)      
#dfw = calc_weights(ef, 'efficient_risk', 0.3, verbose = True) 

OrderedDict([('LKOH', 2.382354e-10), ('NLMK', 1.652831e-10), ('AGRO', 1.386566e-10), ('MAGN', 1.286103e-10), ('TATN', 3.92582e-10), ('SIBN', 1.763884e-10), ('BANEP', 0.0968199988867739), ('SNGSP', 1.1824162e-09), ('POSI', 0.2911879030468951), ('SBER', 3.759347e-10), ('CHMF', 0.6119920951492271), ('ROSN', 1.189969e-10)])
Expected annual return: 125.7%
Annual volatility: 19.9%
Sharpe Ratio: 6.22


In [110]:
dfx = final_sums(dfw, 10000, 50)
dfx = dfx.sort_values("weights", ascending = False)
dfx

,weights
CHMF,6120.0
POSI,2910.0
BANEP,970.0


## All shares

In [54]:
dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]

In [58]:
res = []
for row in dfx.iterrows():
    pos = row[1]
#    print(pos.figi)        
    candles = tink.get_candles(token, pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  tink.get_open_price(candles)
    ticker = tink.figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)


In [65]:
dfp = dfp.dropna(axis = 1)
dfp

,DSKY,IRKT,VSMO,UNAC,TTLK,MGNT,SPBE,ETLN,KZOSP,WUSH,...,MRKS,MSNG,AFKS,PLZL,LKOH,NMTP,TGKA,CNTLP,MTLRP,MRKY
date,,,,,,,,,,,,,,,,,,,,,
2023-10-02,69.52,118.00,43460.0,2.040,1.3300,5535.5,165.2,84.84,32.95,215.00,...,0.7905,3.1095,17.252,11467.5,6716.5,13.170,0.010600,14.44,226.90,0.09900
2023-10-03,69.50,112.90,42500.0,1.990,1.3100,5489.0,161.3,84.58,33.30,212.50,...,0.7800,3.0840,17.300,11166.0,6720.0,12.500,0.010260,14.10,235.00,0.09535
2023-10-04,68.50,113.35,41160.0,1.930,1.2720,5422.5,160.1,87.58,33.06,212.10,...,0.7635,3.0570,17.340,11374.5,6740.0,12.690,0.010500,13.90,236.40,0.09135
2023-10-05,69.16,113.30,43040.0,1.946,1.2860,5458.0,160.6,88.76,32.45,208.88,...,0.7560,3.0250,17.200,11381.0,6706.0,13.000,0.010372,13.80,235.80,0.09270
2023-10-06,69.80,111.20,43100.0,1.900,1.2340,5445.0,158.0,85.92,32.33,205.00,...,0.7605,3.0300,17.370,11371.0,6749.5,12.800,0.010300,13.86,235.05,0.09115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,49.50,67.75,36400.0,1.335,1.0660,7616.5,86.2,97.30,26.50,336.86,...,0.8280,3.3970,19.582,11850.0,7450.0,11.895,0.009340,11.46,324.60,0.09325
2024-03-22,48.70,68.00,37800.0,1.359,1.0920,7604.5,88.8,97.60,26.55,329.00,...,0.8320,3.3955,20.350,11660.0,7420.0,12.110,0.009460,11.50,320.65,0.09480
2024-03-25,48.34,69.30,36860.0,1.338,1.0705,7448.5,88.5,98.70,25.26,324.70,...,0.8290,3.3420,19.899,11424.5,7305.0,11.760,0.009258,11.50,308.40,0.09310


In [87]:
drops = ["GTRK", "SFIN", "ORUP", "UWGN", "RKKE", "KROT", "APTK", "VEON-RX"]
drops = ["SFIN", "GTRK", "NTZL", "LSRG", 'TGKBP']
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

In [90]:
#ef = calc_frontier(dfp, "semicovariance",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf",  "ema_historical_return", span = 180)
ef = calc_frontier(dfp, "ledoit_wolf",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf_constant_correlation",   "ema_historical_return", span = 90)

 
#ef = calc_frontier(dfp, "ledoit_wolf_constant_variance",  "ema_historical_return", span = 120)
#ef = calc_frontier(dfp, "sample_cov")
#weights = ef.max_sharpe()


dfw = calc_weights(ef, 'max_sharpe', 0)      
#dfw = calc_weights(ef, 'efficient_risk', 0.25, verbose = True) 

In [91]:
dfw.sort_values("weights", ascending = False)

,weights
WUSH,0.352181745
YNDX,0.257295837
POSI,0.105724580
CHMF,0.103542834
BANE,0.065952762
OZON,0.041663331
TGKJ,0.029003203
AFKS,0.026541233
OBNEP,0.010448359
RNFT,0.007646117


In [78]:
dfx = final_sums(dfw, 113200)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

inds
x = base[base['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s


In [79]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["buy"] = np.round(dfx.W / (dfx.price * dfx.lot))
dfx["sum"]= dfx.price * dfx.buy * dfx.lot
dfx = dfx.sort_values("W", ascending = False)
dfx.to_csv("t.csv")
dfx

,W,lot,price,buy,sum
WUSH,40440.0,1,329.00000,123.0,40467.000
YNDX,23970.0,1,3885.00000,6.0,23310.000
BANE,12720.0,1,3076.50000,4.0,12306.000
POSI,11760.0,1,2680.00000,4.0,10720.000
CHMF,8570.0,1,1796.60000,5.0,8983.000
TGKBP,5530.0,100000,0.01728,3.0,5184.000
AFKS,1650.0,100,20.35000,1.0,2035.000
FIVE,1640.0,1,2879.00000,1.0,2879.000
OZON,1520.0,1,3541.00000,0.0,0.000
OBNEP,1500.0,1,958.93000,2.0,1917.860


In [74]:
### RiskFolio

In [80]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 5
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

#print_data(dfp)
df_period = dfp.dropna(axis = 1)

for rm in rms:
    print(rm)
    w = riskfolio_weights(df_period, rm, 'MaxRet')
    print(w)

MV
You must convert self.cov to a positive definite matrix
           weights
WUSH   0.025325336
SLAV   0.118773179
TGKJ   0.130692420
TGKBP  0.030718883
OBNEP  0.096482808
GRNT   0.380834030
BANE   0.023779146
ORUP   0.058834047
YNDX   0.041787242
NOMPP  0.083926839
MSV
You must convert self.cov to a positive definite matrix
           weights
WUSH   0.025324385
SLAV   0.118772821
TGKJ   0.130692121
TGKBP  0.030717978
OBNEP  0.096482335
GRNT   0.380834881
BANE   0.023778181
ORUP   0.058833354
YNDX   0.041786425
NOMPP  0.083926299
CVaR
You must convert self.cov to a positive definite matrix
           weights
WUSH   0.025324385
SLAV   0.118772821
TGKJ   0.130692121
TGKBP  0.030717978
OBNEP  0.096482335
GRNT   0.380834881
BANE   0.023778181
ORUP   0.058833354
YNDX   0.041786425
NOMPP  0.083926299
